In [1]:
# Add parent directory
from sys import path as sysPath
sysPath.append('../')

# Add custom modules
sysPath.append('../Machine_Score/')

from Support_Code import general_module as gm
from Support_Code import info_module as im
from Image_Creator import main_image_creator as ic
from Machine_Score import main_machine_score as ms
from Machine_Score import direct_image_compare as dc
from Score_Analysis import main_score_analysis as sa
import main_SIMR as simr

# General modules
from os import path
import numpy as np
from matplotlib import pyplot as plt

sa.test()
gm.test()
im.test()
ic.test()
ms.test()
dc.test()
simr.test()

SA: Hi!  You're in Matthew's Main program for score analysis!
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew Ogden's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
DI: Hi!  You're in direct_image_compare.py
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [2]:
#tDir = '/home/mbo2d/galStuff/spam_data_pl3/587722984435351614/' 
#tDir = '../../targetDir/'
#tDir = '../../1237678620102623480/'
tDir = '../../587722984435351614/'

tInfo = im.target_info_class( targetDir = tDir, printAll=True)

if tInfo.status == False:
    print("WARNING")
else:
    print("Target Good!")

IM: target_info_class.__init__:
	 - targetDir:  ../../587722984435351614/
IM: Target.initTargetDir():
	 - targetDir: ../../587722984435351614/
NEW PATH EXISTS: /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/
	 - targetDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/
	 - infoDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/information/
	 - baseInfoLoc: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/information/base_target_info.json
	 - allInfoLoc: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/information/target_info.json
	 - zooMergerDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/
	 - plotDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/plots/
IM: Target: Opening target info json
Target Good!


In [3]:

print(tInfo.get('zoo_merger_models').get('00001'))

rId = 'r00001'
runArg = gm.inArgClass()
runArg.setArg("printAll",True)

rInfo = tInfo.getRunInfo( rID=rId, rArg = runArg)
rInfo.tInfo = tInfo

if rInfo.status == False:
    print("WARNING")
else:
    print("Run '%s' Good!"%rInfo.get('run_id'))

None
IM: run_info_class.__init__
	 - runDir:  /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/run_00001/
IM: run.initRunDir
	 - runDir: /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/run_00001/
	 - runDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/run_00001/
	 - ptsDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/run_00001/particle_files/
	 - imgDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/run_00001/model_images/
	 - miscDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/run_00001/misc_images/
	 - infoLoc: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/run_00001/info.json
	 - baseLoc: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/587722984435351614/zoo_merger_models/run_00001/base_info.json

In [4]:
pLoc = gm.validPath('../param/start_param.json')
print(pLoc,'\n')
startParams = im.group_score_parameter_class(pLoc = pLoc)

if startParams.status:
    params = startParams.group
    gm.pprint(params)
    print('\n')
    print("Good Starting Parameters!")
else:
    print("NOOOOOO")


/mnt/c/Users/ogden/OneDrive/Desktop/galStuff/galaxyJSPAM/param/start_param.json 

{'base_perturbation': {'cmpArg': {'cmpMethod': 'correlation'},
                       'imgArg': {'name': 'default',
                                  'pType': 'default'},
                       'name': 'base_perturbation',
                       'scoreType': 'perturbation',
                       'simArg': {'nPts': '100k',
                                  'name': '100k'}},
 'base_score': {'cmpArg': {'cmpMethod': 'correlation'},
                'imgArg': {'name': 'default',
                           'pType': 'default'},
                'name': 'base_score',
                'scoreType': 'target',
                'simArg': {'nPts': '100k',
                           'name': '100k'},
                'targetName': 'zoo'}}


Good Starting Parameters!


In [5]:
def perturbation_compare( rInfo, param, args ):
    
    printBase = args.printBase
    printAll = args.printAll
    
    # Base info
    pName = param['name']
    mName = param['imgArg']['name']
    
    if printBase: print('MS: perturbation_compare: %s'%pName)
    
    if printAll:
        im.tabprint(' paramName: %s'%pName)
        im.tabprint(' modelName: %s'%mName)
    
    # Exit if invalid request
    if type(pName) == type(None) and type(mName) == type(None):
        if printBase:
                print("MS: Error: target_image_compare: Bad param given")
        return None
    
    # GET MODEL IMAGE    
    mImg = None
    mLoc = None
    
    # Create dict for storing model images if needed later       
    if rInfo.get('modelImg') == None:
        rInfo.modelImg = {}
        
    # Check if in rInfo has model Image already
    if type( rInfo.modelImg.get(mName) ) != type(None):
        mImg = rInfo.modelImg.get(mName)
    
    # Else, have rInfo retrieve img location
    else:
        mImg = rInfo.getModelImg(mName)
        rInfo.modelImg[mName] = mImg
    
    # Check if valid image
    if type(mImg) == type(None): 
        if printBase: 
            print("MS: Error: target_image_compare: failed to load model image")
            gm.tabprint("runId: %s"%rInfo.get('run_id'))
            gm.tabprint("model: %s"%mName)
        return None
        
    elif printAll: print("MS: perturbation_compare: Read model image: %s"%mName)
        
        
    # GET INIT IMAGE    
    iImg = None
    iLoc = None
    
    # Create dict for storing init images if needed later    
    if rInfo.get('initImg') == None:
        rInfo.initImg = {}
        
    # Check if in rInfo has model Image already
    if type( rInfo.initImg.get(mName) ) != type(None):
        iImg = rInfo.initImg.get(mName)
    
    # Else, have rInfo retrieve img location
    else:
        iImg = rInfo.getModelImg(mName,initImg=True)
        rInfo.initImg[mName] = iImg
        
    if type(iImg) == type(None): 
        if printBase: 
            print("MS: Error: target_image_compare: failed to load init image")
            gm.tabprint("runId: %s"%rInfo.get('run_id'))
            gm.tabprint("model: %s"%mName)
        return None
        
    elif printAll: print("MS: perturbation_compare: Read init image: %s"%mName)
    
    # Create score and add to rInfo
    
    score = dc.createScore( mImg, iImg, param['cmpArg'] )
    newScore = rInfo.addScore( name = pName, score=score )
    
    return score

    
ms.set_test_func( perturbation_compare )

newScoreArg = gm.inArgClass()
newScoreArg.setArg('rInfo',rInfo)
newScoreArg.setArg('overWrite',True)
newScoreArg.setArg('scoreParams',params)
newScoreArg.setArg('printAll',True)
#ms.MS_Run( printBase = True, printAll = True, rInfo = rInfo, arg = newScoreArg, params = params )
simr.simr_run( arg = newScoreArg, )

print('\n')
gm.pprint(rInfo.get('machine_scores'))

SIMR.pipelineRun: Inputs
	 - rDir: None
	 - rInfo: <class 'NoneType'>
IM: run_info_class.__init__
	 - runDir:  None
IM: WARNING: run_info_class.__init__: Invalid run dir
	 - runDir: - None
SIMR.pipelineRun: 
	 - rInfo:  <Support_Code.info_module.run_info_class object at 0x7f1fad52da30>
SIMR.pipelineRun: WARNING: runInfo bad


{'base0_bcorr0': 0.6641861873885665,
 'base0_bcorr1': 0.5711423657294602,
 'base0_bcorr2': 0.6404602265891284,
 'base0_bcorr3': 0.6404418317056928,
 'base0_corr': 0.7376331544200321,
 'base0_diff': 0.9468803106569776,
 'base0_diff_scale': 0.4688031065697763,
 'base0_diff_scale1': 0.8937606213139553,
 'base0_diff_scale2': 0.8229343688565922,
 'base0_overlap0': 0.5054912919157786,
 'base0_overlap1': 0.3655356644471911,
 'base0_overlap2': 0.4603863940468086,
 'base0_overlap3': 0.46573162608245133,
 'base_perturbation': 0.7167233715191668,
 'base_score': 0.7376331544200321}


In [6]:
# Run through all of target now

newScoreTargetArg = gm.inArgClass()
newScoreTargetArg.setArg('tInfo',tInfo)
newScoreTargetArg.setArg('overWrite',True)
newScoreTargetArg.setArg('scoreParams',params)
newScoreTargetArg.setArg('newScore',True)

#simr.simr_target(arg=newScoreTargetArg)
print('DONE')

DONE


In [7]:
print("Hi")

Hi
